In [1]:
import pandas as pd
import json
import os
import requests
from datetime import datetime, time

In [2]:

file1 = 'rawdata/불법주정차 신고현황(21년9월29일_22년10월31일).csv'
file2 = 'rawdata/불법주정차 신고현황(22.11월~23.10월).csv'
file3 = 'rawdata/불법주정차 신고현황(23년11월1일_24년3월13일).csv'

df_reports_1 = pd.read_csv(file1, encoding='utf-8')
df_reports_2 = pd.read_csv(file2, encoding='utf-8')
df_reports_3 = pd.read_csv(file3, encoding='utf-8')


In [3]:
display(df_reports_1.head())

,민원접수일,민원접수시간,주소,경도,위도
0,20210929,00:05:00,서울 강서구 화곡로66길 130,126.859911,37.555537
1,20210929,00:38:00,중구 다산로48길 33 단우물어린이집앞,127.018013,37.567483
2,20210929,00:38:00,서울특별시 중구 마장로 19,127.012793,37.569183
3,20210929,00:50:00,서울특별시 강서구 까치산로15길 21,126.847125,37.547040
4,20210929,01:15:00,서울특별시 중구 다산로 293,127.015703,37.569085


In [44]:
print(df_reports_1.shape)
print(df_reports_2.shape)
print(df_reports_3.shape)

(1110253, 5)
(1466617, 5)
(493658, 5)


In [4]:
file4 = 'rawdata/서울시 공영주차장 안내 정보.csv'
df_park_info = pd.read_csv(file4, encoding='cp949')

In [5]:
df_reports = pd.concat([df_reports_3, df_reports_1, df_reports_2], axis=0)


In [6]:
display(df_park_info.columns)

Index(['주차장코드', '주차장명', '주소', '주차장 종류', '주차장 종류명', '운영구분', '운영구분명', '전화번호',
       '주차현황 정보 제공여부', '주차현황 정보 제공여부명', '총 주차면', '유무료구분', '유무료구분명', '야간무료개방여부',
       '야간무료개방여부명', '평일 운영 시작시각(HHMM)', '평일 운영 종료시각(HHMM)', '주말 운영 시작시각(HHMM)',
       '주말 운영 종료시각(HHMM)', '공휴일 운영 시작시각(HHMM)', '공휴일 운영 종료시각(HHMM)',
       '최종데이터 동기화 시간', '토요일 유,무료 구분', '토요일 유,무료 구분명', '공휴일 유,무료 구분',
       '공휴일 유,무료 구분명', '월 정기권 금액', '노상 주차장 관리그룹번호', '기본 주차 요금',
       '기본 주차 시간(분 단위)', '추가 단위 요금', '추가 단위 시간(분 단위)', '버스 기본 주차 요금',
       '버스 기본 주차 시간(분 단위)', '버스 추가 단위 시간(분 단위)', '버스 추가 단위 요금', '일 최대 요금',
       '위도', '경도'],
      dtype='object')

In [7]:

# Rename columns
df_park_info = df_park_info.rename(columns={
    '총 주차면': '총주차면',
    '유무료구분': '평일유료',
    '평일 운영 시작시각(HHMM)': '평일시작',
    '평일 운영 종료시각(HHMM)': '평일종료',
    '주말 운영 시작시각(HHMM)': '토요일시작',
    '주말 운영 종료시각(HHMM)': '토요일종료',
    '공휴일 운영 시작시각(HHMM)': '공휴일시작',
    '공휴일 운영 종료시각(HHMM)': '공휴일종료',
    '토요일 유,무료 구분': '토요일유료',
    '공휴일 유,무료 구분': '공휴일유료'
})

# Define the desired column order
new_columns = ['주소', '총주차면', '평일유료', '토요일유료', '공휴일유료', '평일시작', '평일종료', '토요일시작', '토요일종료', '공휴일시작', '공휴일종료', '위도', '경도']

# Reorder the columns in the DataFrame
df_parks = df_park_info[new_columns]

display(df_parks.head())
# display(df_parks.tail())


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
0,도봉구 창동 24-0,71.0,Y,N,N,900,1900,900,1900,900,1900,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,0,2400,0,2400,0,2400,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,0,2400,0,2400,0,2400,NaN,NaN
3,영등포구 신길동 184-3,98.0,Y,N,N,0,2400,0,2400,0,2400,37.515205,126.915116
4,영등포구 당산동3가 385-0,190.0,Y,N,N,0,2400,0,2400,0,2400,NaN,NaN


In [8]:
def parse_time(value):
    """Convert an integer or string time (HHMM or HH:MM:SS) to a datetime.time object."""
    if pd.isna(value):
        return None

    # build a zero-padded string
    time_str = str(int(value)).zfill(4) if isinstance(value, (int, float)) else str(value)

    # special-case “2400” or “24:00[:00]” as midnight
    if time_str in ("2400", "24:00", "24:00:00"):
        return time(0, 0)

    try:
        # Try parsing as HHMM
        return pd.to_datetime(time_str, format="%H%M").time()
    except ValueError:
        # Fallback to general parser (HH:MM:SS etc.)
        try:
            return pd.to_datetime(time_str).time()
        except Exception:
            return None

In [9]:
time_columns = ['평일시작', '평일종료', '토요일시작', '토요일종료', '공휴일시작', '공휴일종료']
for col in time_columns:
    df_parks[col] = df_parks[col].apply(parse_time).copy()

display(df_parks.head())

/var/folders/pf/y0k140gn11d95ybm0dgtk6m80000gn/T/ipykernel_14018/50185423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parks[col] = df_parks[col].apply(parse_time).copy()


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
3,영등포구 신길동 184-3,98.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.515205,126.915116
4,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


In [32]:
# Find rows with missing values in columns except '위도' or '경도'
missing_rows = df_parks[df_parks.drop(columns=['위도', '경도']).isnull().any(axis=1)]

# Display rows with missing values
display(missing_rows)

# Drop rows with missing values in columns except '위도' or '경도'
df_parks_dropped = df_parks.drop(missing_rows.index)

# Display the updated DataFrame
display(df_parks_dropped)

,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
7688,종로구 명륜4가 105-3,NaN,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8058,용산구 남영동 71-2,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8059,용산구 남영동 71-2,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8073,송파구 가락동 156-11,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8080,송파구 문정동 640-9,NaN,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8278,송파구 풍납동 306-25,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8279,송파구 풍납동 222-75,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8280,송파구 풍납동 292-33,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8281,송파구 풍납동 140-3,NaN,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
3,영등포구 신길동 184-3,98.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.515205,126.915116
4,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8505,강북구 미아동 748-2,95.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8506,강북구 수유동 49-7,80.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8507,종로구 구기동 139-9,40.0,Y,N,N,08:00:00,18:00:00,08:00:00,18:00:00,08:00:00,18:00:00,NaN,NaN
8508,구로구 구로동 1-4,65.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


In [33]:
df_parks_dropped = df_parks_dropped.reset_index(drop=True)
# Display the updated DataFrame
display(df_parks_dropped)


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
3,영등포구 신길동 184-3,98.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.515205,126.915116
4,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8326,강북구 미아동 748-2,95.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8327,강북구 수유동 49-7,80.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8328,종로구 구기동 139-9,40.0,Y,N,N,08:00:00,18:00:00,08:00:00,18:00:00,08:00:00,18:00:00,NaN,NaN
8329,구로구 구로동 1-4,65.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


In [34]:
# Find rows in df_parks_dropped with any missing values
df_parks_missing = df_parks_dropped[df_parks_dropped.isnull().any(axis=1)]

# Drop these rows from df_parks_dropped
df_parks_dropped = df_parks_dropped.drop(df_parks_missing.index)

# Display the DataFrame with missing rows and the cleaned DataFrame
display(df_parks_missing)
display(df_parks_dropped)

,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,NaN,NaN
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
4,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
5,영등포구 대림동 780-0,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8326,강북구 미아동 748-2,95.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8327,강북구 수유동 49-7,80.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN
8328,종로구 구기동 139-9,40.0,Y,N,N,08:00:00,18:00:00,08:00:00,18:00:00,08:00:00,18:00:00,NaN,NaN
8329,구로구 구로동 1-4,65.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,NaN,NaN


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
3,영등포구 신길동 184-3,98.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.515205,126.915116
6,동대문구 휘경동 308-32,78.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,37.588069,127.058093
10,강남구 압구정동 428-0,344.0,Y,N,N,09:00:00,21:00:00,09:00:00,21:00:00,09:00:00,21:00:00,37.527017,127.026471
12,강남구 개포동 1273-0,194.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.475646,127.052365
16,구로구 구로동 810-3,91.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.485432,126.901243
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8305,강북구 수유동 535-76,1.0,Y,N,N,09:00:00,19:00:00,09:00:00,15:00:00,09:00:00,18:00:00,37.643145,127.002927
8308,강서구 화곡동 809-26,203.0,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.534312,126.861058
8310,강서구 화곡동 890-0,248.0,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.529046,126.848861
8318,양천구 목동 905-32,48.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,37.533440,126.876610


In [35]:
# Function to get coordinates for a given address
def get_coordinates(address):
    apiurl = "https://api.vworld.kr/req/address?"
    apikey =  "D9C1CFC9-0FD3-3304-A3E7-131C33A91190"

    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address,
        "format": "json",
        "type": "parcel",
        "key": apikey
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['response']['status'] == 'OK':
            point_info = data['response']['result']['point']
            return point_info['x'], point_info['y']
    return None, None


In [36]:
# Apply the function to the '주소' column and store the results
df_parks_missing[['위도', '경도']] = df_parks_missing['주소'].apply(lambda addr: pd.Series(get_coordinates(addr)))

# Display the updated dataframe
df_parks_missing.head()

,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,127.05100768455492,37.6534847540488
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.05798586176407,37.64367511436389
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.03510376366461,37.56996805245046
4,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89577762856109,37.52554507178421
5,영등포구 대림동 780-0,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89444474098582,37.49923824170827


In [ ]:
df_parks_missing

,주소,위도,경도
0,도봉구 창동 24-0,127.05100768455492,37.6534847540488
1,노원구 상계동 770-2,127.05798586176407,37.64367511436389
2,성동구 마장동 463-2,127.03510376366461,37.56996805245046
3,영등포구 당산동3가 385-0,126.89577762856109,37.52554507178421
4,영등포구 대림동 780-0,126.89444474098582,37.49923824170827
...,...,...,...
1054,강북구 미아동 791-1360,127.01337142398819,37.626166339528766
1055,강북구 미아동 791-1429,127.01571664763247,37.62589357898619
1056,강북구 미아동 748-2,127.01847091587678,37.622737640361635
1057,구로구 구로동 1-4,126.89159944589176,37.50731316416754


In [37]:
df_parks_combined = pd.concat([df_parks_missing, df_parks_dropped], ignore_index=True)

# Display the combined DataFrame
display(df_parks_combined)

,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,127.05100768455492,37.6534847540488
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.05798586176407,37.64367511436389
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.03510376366461,37.56996805245046
3,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89577762856109,37.52554507178421
4,영등포구 대림동 780-0,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89444474098582,37.49923824170827
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8326,강북구 수유동 535-76,1.0,Y,N,N,09:00:00,19:00:00,09:00:00,15:00:00,09:00:00,18:00:00,37.643145,127.002927
8327,강서구 화곡동 809-26,203.0,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.534312,126.861058
8328,강서구 화곡동 890-0,248.0,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.529046,126.848861
8329,양천구 목동 905-32,48.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,37.53344,126.87661


In [41]:
# Display rows with missing values
missing_rows_combined = df_parks_combined[df_parks_combined.isnull().any(axis=1)]
print(df_parks_combined.isnull().sum())
display(missing_rows_combined)

# Drop rows with missing values
df_parks_cleaned = df_parks_combined.dropna()

# Display the cleaned DataFrame
display(df_parks_cleaned)

주소        0
총주차면      0
평일유료      0
토요일유료     0
공휴일유료     0
평일시작      0
평일종료      0
토요일시작     0
토요일종료     0
공휴일시작     0
공휴일종료     0
위도       54
경도       54
dtype: int64


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
73,강북구 미아동 137-19,1.0,Y,N,N,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,None,None
92,강북구 수유동 55-53,116.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,None,None
111,종로구 장사동 116-3,1.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,None,None
127,종로구 창신동 37-16,1.0,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,None,None
128,종로구 인사동 283-0,1.0,Y,N,Y,10:00:00,22:00:00,09:00:00,17:00:00,00:00:00,00:00:00,None,None
129,종로구 공평동 283-0,1.0,Y,N,Y,10:00:00,22:00:00,09:00:00,17:00:00,00:00:00,00:00:00,None,None
144,종로구 창신동 67-2,1.0,N,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,None,None
156,노원구 상계동 111-568,1.0,Y,N,N,09:00:00,22:00:00,09:00:00,22:00:00,09:00:00,22:00:00,None,None
263,강서구 화곡동 980-19,1.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,None,None
268,강서구 화곡동 980-19,1.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,None,None


,주소,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,위도,경도
0,도봉구 창동 24-0,71.0,Y,N,N,09:00:00,19:00:00,09:00:00,19:00:00,09:00:00,19:00:00,127.05100768455492,37.6534847540488
1,노원구 상계동 770-2,177.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.05798586176407,37.64367511436389
2,성동구 마장동 463-2,54.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.03510376366461,37.56996805245046
3,영등포구 당산동3가 385-0,190.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89577762856109,37.52554507178421
4,영등포구 대림동 780-0,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,126.89444474098582,37.49923824170827
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8326,강북구 수유동 535-76,1.0,Y,N,N,09:00:00,19:00:00,09:00:00,15:00:00,09:00:00,18:00:00,37.643145,127.002927
8327,강서구 화곡동 809-26,203.0,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.534312,126.861058
8328,강서구 화곡동 890-0,248.0,Y,Y,Y,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,37.529046,126.848861
8329,양천구 목동 905-32,48.0,Y,N,Y,09:00:00,19:00:00,09:00:00,15:00:00,00:00:00,00:00:00,37.53344,126.87661


In [ ]:
df_parks_cleaned.to_csv('parks.csv', index=False, encoding='utf-8-sig')

In [43]:
df_reports

,민원접수일,민원접수시간,주소,경도,위도
0,20231101,00:00:00,서울특별시 성동구 사근동길 45,127.045952,37.560972
1,20231101,00:03:00,구로구 가마산로12길 18 앞쪽 고가차도 밑,126.882717,37.492723
2,20231101,00:04:00,공/남부순환로11길 55,126.814655,37.555981
3,20231101,00:07:00,아차산로57길 4 CU구의중앙점 앞,127.088326,37.538708
4,20231101,00:09:00,방학동717-2 [ 도봉로150다길 28] ~ 방학동717-3 주차장 입구 및 전체...,127.044821,37.667286
...,...,...,...,...,...
1466612,20231031,23:55:00,서울특별시 용산구 한남대로20길 47-10,127.009894,37.535221
1466613,20231031,23:56:00,서울특별시 양천구 목동중앙서로7나길 11,126.865180,37.534377
1466614,20231031,23:59:00,서울특별시 서초구 반포대로 304 금정빌딩,127.001042,37.506165
1466615,20231031,23:59:00,서울특별시 서초구 신반포로23길 76 잠원시티아파트,127.005346,37.511840


In [45]:
# Find rows with missing values in columns except '위도' or '경도'
missing_rows = df_reports[df_reports.drop(columns=['위도', '경도']).isnull().any(axis=1)]

# Display rows with missing values
display(missing_rows)

# Drop rows with missing values in columns except '위도' or '경도'
df_reports_dropped = df_reports.drop(missing_rows.index)

# Display the updated DataFrame
display(df_reports_dropped)

,민원접수일,민원접수시간,주소,경도,위도
2727,20231101,18:49:00,NaN,127.067222,37.664983
3092,20231101,19:51:00,NaN,127.067170,37.664344
3136,20231101,20:02:00,NaN,126.906070,37.442495
20860,20231106,07:34:00,NaN,127.059652,37.669789
27235,20231107,22:04:00,NaN,127.071881,37.645576
...,...,...,...,...,...
1465939,20231031,20:04:00,NaN,0.000000,0.000000
1465987,20231031,20:17:00,NaN,0.000000,0.000000
1466103,20231031,20:45:00,NaN,127.063964,37.645916
1466558,20231031,23:22:00,NaN,0.000000,0.000000


,민원접수일,민원접수시간,주소,경도,위도
0,20231101,00:00:00,서울특별시 성동구 사근동길 45,127.045952,37.560972
1,20231101,00:03:00,구로구 가마산로12길 18 앞쪽 고가차도 밑,126.882717,37.492723
2,20231101,00:04:00,공/남부순환로11길 55,126.814655,37.555981
3,20231101,00:07:00,아차산로57길 4 CU구의중앙점 앞,127.088326,37.538708
4,20231101,00:09:00,방학동717-2 [ 도봉로150다길 28] ~ 방학동717-3 주차장 입구 및 전체...,127.044821,37.667286
...,...,...,...,...,...
1466612,20231031,23:55:00,서울특별시 용산구 한남대로20길 47-10,127.009894,37.535221
1466613,20231031,23:56:00,서울특별시 양천구 목동중앙서로7나길 11,126.865180,37.534377
1466614,20231031,23:59:00,서울특별시 서초구 반포대로 304 금정빌딩,127.001042,37.506165
1466615,20231031,23:59:00,서울특별시 서초구 신반포로23길 76 잠원시티아파트,127.005346,37.511840


In [46]:
# Find rows in df_reports_dropped with any missing values
df_reports_missing = df_reports_dropped[df_reports_dropped.isnull().any(axis=1)]

# Drop these rows from df_reports_dropped
df_reports_dropped = df_reports_dropped.drop(df_reports_missing.index)

# Display the DataFrame with missing rows and the cleaned DataFrame
display(df_reports_missing)
display(df_reports_dropped)

,민원접수일,민원접수시간,주소,경도,위도
59372,20221117,22:00:00,은천로103-1,NaN,NaN
122752,20221204,13:04:00,삼성동 8-2 ~ 9 / 치즈샵 앞 우회전 차로 불법 주정차,NaN,NaN
148609,20221212,14:03:00,벚꽃로56길 92,NaN,NaN
174185,20221220,16:40:00,삼성동 10-25와 롯데캐슬 프레미어 106동 사이 도로변 인도 옆 불법 주정차,NaN,NaN
188120,20221224,19:26:00,까치산로4다길 18,NaN,NaN
212396,20221231,17:30:00,개포동 141 아이파크 157동 앞 대로변[양재대로방면 버스정류장 양뱡향]-5번버스...,NaN,NaN
214848,20230101,12:31:00,노량진로23가길23,NaN,NaN
283668,20230120,19:23:00,(청룡동)남부순환로214길 초입,NaN,NaN
327354,20230202,22:45:00,화3/ 화곡로19길38,NaN,NaN
347196,20230208,08:26:00,\t강남구. 테헤란도 440( 대치동 892) 와 대치롯데캐슬 아파트 (삼성로 85...,NaN,NaN


,민원접수일,민원접수시간,주소,경도,위도
0,20231101,00:00:00,서울특별시 성동구 사근동길 45,127.045952,37.560972
1,20231101,00:03:00,구로구 가마산로12길 18 앞쪽 고가차도 밑,126.882717,37.492723
2,20231101,00:04:00,공/남부순환로11길 55,126.814655,37.555981
3,20231101,00:07:00,아차산로57길 4 CU구의중앙점 앞,127.088326,37.538708
4,20231101,00:09:00,방학동717-2 [ 도봉로150다길 28] ~ 방학동717-3 주차장 입구 및 전체...,127.044821,37.667286
...,...,...,...,...,...
1466612,20231031,23:55:00,서울특별시 용산구 한남대로20길 47-10,127.009894,37.535221
1466613,20231031,23:56:00,서울특별시 양천구 목동중앙서로7나길 11,126.865180,37.534377
1466614,20231031,23:59:00,서울특별시 서초구 반포대로 304 금정빌딩,127.001042,37.506165
1466615,20231031,23:59:00,서울특별시 서초구 신반포로23길 76 잠원시티아파트,127.005346,37.511840


In [48]:
df_reports_missing.shape

(27, 5)

In [49]:
df_reports_cleaned = df_reports_dropped.copy()


In [ ]:
df_reports_cleaned.to_csv('reports.csv', index=False, encoding='utf-8-sig')

In [51]:
missing_values = df_reports_cleaned.isnull().sum()
print(missing_values)

민원접수일     0
민원접수시간    0
주소        0
경도        0
위도        0
dtype: int64


In [52]:
df_reports_cleaned.head()

,민원접수일,민원접수시간,주소,경도,위도
0,20231101,00:00:00,서울특별시 성동구 사근동길 45,127.045952,37.560972
1,20231101,00:03:00,구로구 가마산로12길 18 앞쪽 고가차도 밑,126.882717,37.492723
2,20231101,00:04:00,공/남부순환로11길 55,126.814655,37.555981
3,20231101,00:07:00,아차산로57길 4 CU구의중앙점 앞,127.088326,37.538708
4,20231101,00:09:00,방학동717-2 [ 도봉로150다길 28] ~ 방학동717-3 주차장 입구 및 전체...,127.044821,37.667286


In [53]:
pip install holidays

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.3/932.3 kB 33.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import holidays

# 1) Make sure '날짜' is a datetime
#    yours is in YYYYMMDD integer or string form
df_reports_cleaned['날짜'] = pd.to_datetime(df_reports_cleaned['날짜'].astype(str), format='%Y%m%d')

# 2) Load Korean public holidays for all relevant years
years = df_reports_cleaned['날짜'].dt.year.unique().tolist()
kr_holidays = holidays.KR(years=years)

# 3) Define a function to classify each date
def classify_day(d):
    wd = d.weekday()  # Mon=0 … Sat=5, Sun=6
    if wd == 5:
        return 'Saturday'
    # treat Sunday or any official holiday as Holiday
    elif wd == 6 or d in kr_holidays:
        return 'Holiday'
    else:
        return 'Weekday'

# 4) Apply it
df_reports_cleaned['요일'] = df_reports_cleaned['날짜'].apply(classify_day)

# 5) (Optional) Quick sanity‐check
print(df_reports_cleaned[['날짜','요일']].drop_duplicates().sort_values('날짜').head(10))